In [1]:
%load_ext autoreload
%autoreload 2

This notebook highlights practical use cases for the FLamby integration into Fed-BioMed. A few datasets are here tested:
1. IXI Tiny
2. Heart-Disease
3. TCGA-BRCA
4. Synthetic
5. ISIC

For one FLamby dataset (LIDC-IDRI), we noticed conflicts over few packages including Torch, which cannot be easily resolved by changing package versions but need changes in the code (either on FLamby or Fed-BioMed side). This situation shows a certain complexity regarding the interoperability and has to be taken into account for future datasets that will be part of the suite.

### 1. Fed-IXI

In [10]:
from fedbiomed.common.training_plans import TorchTrainingPlan
from torch.optim import AdamW

from flamby.datasets.fed_ixi import (Baseline as ixi_baseline,
                                     BaselineLoss as ixi_baseline_loss,
                                     Optimizer as ixi_optimizer,
                                     BATCH_SIZE as ixi_batch_size,
                                     LR as ixi_lr)

class UNetTrainingPlan(TorchTrainingPlan):
    # Init of UNetTrainingPlan
    def __init__(self, model_args: dict = {}):
        super(UNetTrainingPlan, self).__init__(model_args)
        
        self.model = ixi_baseline() # UNet model
        self.loss = ixi_baseline_loss() # Dice loss
        
        deps = ['from torch.optim import AdamW',
               'from flamby.datasets.fed_ixi import (Baseline as ixi_baseline,\
                BaselineLoss as ixi_baseline_loss,\
                Optimizer as ixi_optimizer)',]
        self.add_dependency(deps)
        
        self.optimizer = ixi_optimizer(self.parameters()) # AdamW
    
    def training_step(self, img, target):
        #this function must return the loss to backward it 
        output = self.model(img)
        loss = self.loss(output, target)
        return loss

In [11]:
training_args = {
    'batch_size': ixi_batch_size,
    'lr': ixi_lr,
    'epochs': 1,
}

*train_transform_flamby* key in **training_args** can optionally be used to perform extra transformations on a flamby dataset.

As a reminder, flamby datasets are already internally handling a transformation through their dataloader (this internal transform
is the one officially used for the flamby benchmark). Thus, one should check what is already performed on the flamby side before
adding transforms through the researcher.

*train_transform_flamby* has to be defined as a list containing two elements:
- the first is the imports needed to perform the transformation
- the second is the Compose object that will be used to input the transform parameter of the flamby dataset federated class

Example:
```python
training_args = {
    ...,
    'train_transform_flamby':["from monai.transforms import (Compose, NormalizeIntensity, Resize,)",
                         "Compose([Resize((48,60,48)), NormalizeIntensity()])"]
}
```

In [12]:
from fedbiomed.researcher.experiment import Experiment
from fedbiomed.researcher.aggregators.fedavg import FedAverage

tags =  ['ixi']
num_rounds = 1

exp = Experiment(tags=tags,
                 model_class=UNetTrainingPlan,
                 training_args=training_args,
                 round_limit=num_rounds,
                 aggregator=FedAverage(),
                )

2022-07-21 18:17:29,387 fedbiomed INFO - Searching dataset with data tags: ['ixi'] for all nodes
2022-07-21 18:17:39,402 fedbiomed INFO - Node selected for training -> node_15a68d33-1a5d-408a-959c-042c1287feab
2022-07-21 18:17:39,403 fedbiomed INFO - Node selected for training -> node_089d07d3-095a-48d7-b49a-f55edc79307e
2022-07-21 18:17:39,406 fedbiomed INFO - Checking data quality of federated datasets...
2022-07-21 18:17:39,434 fedbiomed DEBUG - Model file has been saved: /home/ayed/fedbiomed/var/experiments/Experiment_0005/my_model_4c7979d6-1c2a-40a6-9e01-a9858f7db514.py
2022-07-21 18:17:39,495 fedbiomed DEBUG - upload (HTTP POST request) of file /home/ayed/fedbiomed/var/experiments/Experiment_0005/my_model_4c7979d6-1c2a-40a6-9e01-a9858f7db514.py successful, with status code 201
2022-07-21 18:17:39,586 fedbiomed DEBUG - upload (HTTP POST request) of file /home/ayed/fedbiomed/var/experiments/Experiment_0005/aggregated_params_init_383ed86f-a4e1-4746-8d62-cde76f18c267.pt successful, w

In [13]:
exp.run()

2022-07-21 18:17:53,599 fedbiomed INFO - Sampled nodes in round 0 ['node_15a68d33-1a5d-408a-959c-042c1287feab', 'node_089d07d3-095a-48d7-b49a-f55edc79307e']
2022-07-21 18:17:53,601 fedbiomed INFO - Sending request 
					 To: node_15a68d33-1a5d-408a-959c-042c1287feab 
					 Request: : Perform training with the arguments: {'researcher_id': 'researcher_5c9cf95a-5371-48a9-9b6f-0152737ee6f2', 'job_id': 'bb037630-2731-4a50-88c0-707137b26d80', 'training_args': scheme:
{'lr': {'rules': [<class 'float'>, <function TrainingArgs._lr_hook at 0x7f7f6c629310>], 'required': False}, 'batch_size': {'rules': [<class 'int'>], 'required': False}, 'epochs': {'rules': [<class 'int'>], 'required': False}, 'dry_run': {'rules': [<class 'bool'>], 'required': False}, 'batch_maxnum': {'rules': [<class 'int'>], 'required': False}, 'test_ratio': {'rules': [<class 'float'>, <function TrainingArgs._test_ratio_hook at 0x7f7f6c6291f0>], 'required': False, 'default': 0.0}, 'test_on_local_updates': {'rules': [<class 'boo

2022-07-21 18:21:33,853 fedbiomed INFO - Downloading model params after training on node_15a68d33-1a5d-408a-959c-042c1287feab - from http://localhost:8844/media/uploads/2022/07/21/node_params_7c19c48f-768d-43a2-bfa1-71ba39f3b99c.pt
2022-07-21 18:21:33,866 fedbiomed DEBUG - upload (HTTP GET request) of file node_params_b4c85b0d-b8d5-4a2b-bd2b-5499f27718c4.pt successful, with status code 200
2022-07-21 18:21:38,578 fedbiomed INFO - TRAINING 
					 NODE_ID: node_089d07d3-095a-48d7-b49a-f55edc79307e 
					 Epoch: 1 | Completed: 160/249 (64%) 
 					 Loss: 0.111232 
					 ---------
2022-07-21 18:21:54,737 fedbiomed INFO - TRAINING 
					 NODE_ID: node_089d07d3-095a-48d7-b49a-f55edc79307e 
					 Epoch: 1 | Completed: 180/249 (72%) 
 					 Loss: 0.099106 
					 ---------
2022-07-21 18:22:11,078 fedbiomed INFO - TRAINING 
					 NODE_ID: node_089d07d3-095a-48d7-b49a-f55edc79307e 
					 Epoch: 1 | Completed: 200/249 (80%) 
 					 Loss: 0.087368 
					 ---------
2022-07-21 18:22:27,513 fedbiomed 

1

### 2. Fed-Heart-Disease

In [24]:
from flamby.datasets.fed_heart_disease import (Baseline as hd_baseline,
                                               BaselineLoss as hd_baseline_loss,
                                               Optimizer as hd_optimizer,
                                               BATCH_SIZE as hd_batch_size,
                                               LR as hd_lr)

from fedbiomed.common.training_plans import TorchTrainingPlan
from torch.optim import Adam

class FedHeartTrainingPlan(TorchTrainingPlan):
    def __init__(self, model_args: dict = {}):
        super(FedHeartTrainingPlan, self).__init__(model_args)
        
        self.model = hd_baseline()
        self.loss = hd_baseline_loss()
        
        deps = ['from torch.optim import Adam',
                'from flamby.datasets.fed_heart_disease import (Baseline as hd_baseline,\
                BaselineLoss as hd_baseline_loss,\
                Optimizer as hd_optimizer)']
        self.add_dependency(deps)
        
        self.optimizer = hd_optimizer(self.parameters())
    
    def training_step(self, data, target):
        output = self.model(data)
        loss = self.loss(output, target)
        return loss

In [25]:
training_args = {
    'batch_size': hd_batch_size,
    'lr': hd_lr,
    'epochs': 5,
}

In [26]:
from fedbiomed.researcher.experiment import Experiment
from fedbiomed.researcher.aggregators.fedavg import FedAverage

tags =  ['hd']
num_rounds = 1

exp = Experiment(tags=tags,
                 model_class=FedHeartTrainingPlan,
                 training_args=training_args,
                 round_limit=num_rounds,
                 aggregator=FedAverage(),
                )

2022-07-20 15:48:45,824 fedbiomed INFO - Searching dataset with data tags: ['hd'] for all nodes
2022-07-20 15:48:55,838 fedbiomed INFO - Node selected for training -> node_8e6bb63e-8a5a-41c1-94a0-3c980435dbb9
2022-07-20 15:48:55,839 fedbiomed INFO - Node selected for training -> node_1eb02619-83fa-4523-8c2d-259ed6835d63
2022-07-20 15:48:55,841 fedbiomed INFO - Checking data quality of federated datasets...
2022-07-20 15:48:55,845 fedbiomed DEBUG - Model file has been saved: /home/ayed/fedbiomed/var/experiments/Experiment_0013/my_model_add47bd3-683b-4ead-a2c3-0d56feb38940.py
2022-07-20 15:48:55,877 fedbiomed DEBUG - upload (HTTP POST request) of file /home/ayed/fedbiomed/var/experiments/Experiment_0013/my_model_add47bd3-683b-4ead-a2c3-0d56feb38940.py successful, with status code 201
2022-07-20 15:48:55,896 fedbiomed DEBUG - upload (HTTP POST request) of file /home/ayed/fedbiomed/var/experiments/Experiment_0013/aggregated_params_init_32b72181-d6fd-4a6f-ad92-11d4fe5ee5cf.pt successful, wi

In [27]:
exp.run()

2022-07-20 15:49:01,774 fedbiomed INFO - Sampled nodes in round 0 ['node_8e6bb63e-8a5a-41c1-94a0-3c980435dbb9', 'node_1eb02619-83fa-4523-8c2d-259ed6835d63']
2022-07-20 15:49:01,777 fedbiomed INFO - Sending request 
					 To: node_8e6bb63e-8a5a-41c1-94a0-3c980435dbb9 
					 Request: : Perform training with the arguments: {'researcher_id': 'researcher_dfd1d299-4cc6-49aa-8623-b8b8b842003c', 'job_id': '254fc2a2-67ff-4011-9742-c8d223e7fd74', 'training_args': scheme:
{'lr': {'rules': [<class 'float'>, <function TrainingArgs._lr_hook at 0x7f5e4f623af0>], 'required': False}, 'batch_size': {'rules': [<class 'int'>], 'required': False}, 'epochs': {'rules': [<class 'int'>], 'required': False}, 'dry_run': {'rules': [<class 'bool'>], 'required': False}, 'batch_maxnum': {'rules': [<class 'int'>], 'required': False}, 'test_ratio': {'rules': [<class 'float'>, <function TrainingArgs._test_ratio_hook at 0x7f5e4f6239d0>], 'required': False, 'default': 0.0}, 'test_on_local_updates': {'rules': [<class 'boo

2022-07-20 15:49:02,265 fedbiomed INFO - TRAINING 
					 NODE_ID: node_1eb02619-83fa-4523-8c2d-259ed6835d63 
					 Epoch: 2 | Completed: 96/199 (46%) 
 					 Loss: 8.573589 
					 ---------
2022-07-20 15:49:02,268 fedbiomed INFO - TRAINING 
					 NODE_ID: node_1eb02619-83fa-4523-8c2d-259ed6835d63 
					 Epoch: 2 | Completed: 128/199 (62%) 
 					 Loss: 3.483988 
					 ---------
2022-07-20 15:49:02,270 fedbiomed INFO - TRAINING 
					 NODE_ID: node_8e6bb63e-8a5a-41c1-94a0-3c980435dbb9 
					 Epoch: 2 | Completed: 96/172 (55%) 
 					 Loss: 5.191084 
					 ---------
2022-07-20 15:49:02,272 fedbiomed INFO - TRAINING 
					 NODE_ID: node_1eb02619-83fa-4523-8c2d-259ed6835d63 
					 Epoch: 2 | Completed: 160/199 (77%) 
 					 Loss: 8.136700 
					 ---------
2022-07-20 15:49:02,274 fedbiomed INFO - TRAINING 
					 NODE_ID: node_8e6bb63e-8a5a-41c1-94a0-3c980435dbb9 
					 Epoch: 2 | Completed: 128/172 (73%) 
 					 Loss: 3.339804 
					 ---------
2022-07-20 15:49:02,280 fedbiomed INFO - TRAINI

2022-07-20 15:49:02,386 fedbiomed INFO - INFO
					 NODE node_8e6bb63e-8a5a-41c1-94a0-3c980435dbb9
					 MESSAGE: results uploaded successfully 
-----------------------------------------------------------------
2022-07-20 15:49:02,389 fedbiomed INFO - INFO
					 NODE node_1eb02619-83fa-4523-8c2d-259ed6835d63
					 MESSAGE: results uploaded successfully 
-----------------------------------------------------------------
2022-07-20 15:49:11,796 fedbiomed INFO - Downloading model params after training on node_8e6bb63e-8a5a-41c1-94a0-3c980435dbb9 - from http://localhost:8844/media/uploads/2022/07/20/node_params_3123f58e-7a24-4599-99f9-df7d3e8e09b6.pt
2022-07-20 15:49:11,806 fedbiomed DEBUG - upload (HTTP GET request) of file node_params_7be40e02-c3d9-48f8-a44f-304a456f1719.pt successful, with status code 200
2022-07-20 15:49:11,808 fedbiomed INFO - Downloading model params after training on node_1eb02619-83fa-4523-8c2d-259ed6835d63 - from http://localhost:8844/media/uploads/2022/07/20/node_

1

2022-07-20 16:08:04,071 fedbiomed INFO - CRITICAL
					 NODE node_1eb02619-83fa-4523-8c2d-259ed6835d63
					 MESSAGE: Node stopped in signal_handler, probably by user decision (Ctrl C)
-----------------------------------------------------------------
2022-07-20 16:08:07,021 fedbiomed INFO - CRITICAL
					 NODE node_8e6bb63e-8a5a-41c1-94a0-3c980435dbb9
					 MESSAGE: Node stopped in signal_handler, probably by user decision (Ctrl C)
-----------------------------------------------------------------
2022-07-20 16:09:05,041 fedbiomed INFO - INFO
					 NODE node_1eb02619-83fa-4523-8c2d-259ed6835d63
					 MESSAGE: Starting task manager
-----------------------------------------------------------------
2022-07-20 16:09:44,297 fedbiomed INFO - INFO
					 NODE node_8e6bb63e-8a5a-41c1-94a0-3c980435dbb9
					 MESSAGE: Starting task manager
-----------------------------------------------------------------


### 3. Fed-TCGA-BRCA

In [1]:
from flamby.datasets.fed_tcga_brca import (Baseline as tc_baseline,
                                           BaselineLoss as tc_baseline_loss,
                                           Optimizer as tc_optimizer,
                                           BATCH_SIZE as tc_batch_size,
                                           LR as tc_lr)
from fedbiomed.common.training_plans import TorchTrainingPlan
from torch.optim import Adam

class FedTcgaBrcaTrainingPlan(TorchTrainingPlan):
    def __init__(self, model_args: dict = {}):
        super(FedTcgaBrcaTrainingPlan, self).__init__(model_args)
        
        self.model = tc_baseline()
        self.loss = tc_baseline_loss()
        
        deps = ['from torch.optim import Adam',
               'from flamby.datasets.fed_tcga_brca import (Baseline as tc_baseline,\
                BaselineLoss as tc_baseline_loss,\
                Optimizer as tc_optimizer)']
        self.add_dependency(deps)
        
        self.optimizer = tc_optimizer(self.parameters())
    
    def training_step(self, data, target):
        output = self.model(data)
        loss = self.loss(output, target)
        return loss

In [2]:
training_args = {
    'batch_size': tc_batch_size,
    'lr': tc_lr,
    'epochs': 5,
}

In [3]:
from fedbiomed.researcher.experiment import Experiment
from fedbiomed.researcher.aggregators.fedavg import FedAverage

tags =  ['tc']
num_rounds = 1

exp = Experiment(tags=tags,
                 model_class=FedTcgaBrcaTrainingPlan,
                 training_args=training_args,
                 round_limit=num_rounds,
                 aggregator=FedAverage(),
                )

2022-07-21 17:16:33,582 fedbiomed INFO - Component environment:
2022-07-21 17:16:33,584 fedbiomed INFO - type = ComponentType.RESEARCHER
2022-07-21 17:16:33,852 fedbiomed INFO - Messaging researcher_5c9cf95a-5371-48a9-9b6f-0152737ee6f2 successfully connected to the message broker, object = <fedbiomed.common.messaging.Messaging object at 0x7f7f6bd6e970>
2022-07-21 17:16:33,900 fedbiomed INFO - Searching dataset with data tags: ['tc'] for all nodes
2022-07-21 17:16:43,909 fedbiomed INFO - Node selected for training -> node_15a68d33-1a5d-408a-959c-042c1287feab
2022-07-21 17:16:43,910 fedbiomed INFO - Node selected for training -> node_089d07d3-095a-48d7-b49a-f55edc79307e
2022-07-21 17:16:43,912 fedbiomed INFO - Checking data quality of federated datasets...
2022-07-21 17:16:43,918 fedbiomed DEBUG - Model file has been saved: /home/ayed/fedbiomed/var/experiments/Experiment_0003/my_model_8117b67b-a38d-4050-9061-038ed1de7119.py
2022-07-21 17:16:44,088 fedbiomed DEBUG - upload (HTTP POST requ

In [4]:
exp.run()

2022-07-21 17:16:49,523 fedbiomed INFO - Sampled nodes in round 0 ['node_15a68d33-1a5d-408a-959c-042c1287feab', 'node_089d07d3-095a-48d7-b49a-f55edc79307e']
2022-07-21 17:16:49,525 fedbiomed INFO - Sending request 
					 To: node_15a68d33-1a5d-408a-959c-042c1287feab 
					 Request: : Perform training with the arguments: {'researcher_id': 'researcher_5c9cf95a-5371-48a9-9b6f-0152737ee6f2', 'job_id': '07a8e07e-65f3-4662-8620-adfd6630a7c5', 'training_args': scheme:
{'lr': {'rules': [<class 'float'>, <function TrainingArgs._lr_hook at 0x7f7f6c629310>], 'required': False}, 'batch_size': {'rules': [<class 'int'>], 'required': False}, 'epochs': {'rules': [<class 'int'>], 'required': False}, 'dry_run': {'rules': [<class 'bool'>], 'required': False}, 'batch_maxnum': {'rules': [<class 'int'>], 'required': False}, 'test_ratio': {'rules': [<class 'float'>, <function TrainingArgs._test_ratio_hook at 0x7f7f6c6291f0>], 'required': False, 'default': 0.0}, 'test_on_local_updates': {'rules': [<class 'boo

2022-07-21 17:16:50,000 fedbiomed INFO - TRAINING 
					 NODE_ID: node_089d07d3-095a-48d7-b49a-f55edc79307e 
					 Epoch: 2 | Completed: 96/248 (38%) 
 					 Loss: 0.307423 
					 ---------
2022-07-21 17:16:50,001 fedbiomed INFO - TRAINING 
					 NODE_ID: node_15a68d33-1a5d-408a-959c-042c1287feab 
					 Epoch: 2 | Completed: 32/156 (20%) 
 					 Loss: 0.418945 
					 ---------
2022-07-21 17:16:50,031 fedbiomed INFO - TRAINING 
					 NODE_ID: node_089d07d3-095a-48d7-b49a-f55edc79307e 
					 Epoch: 2 | Completed: 128/248 (50%) 
 					 Loss: 0.089709 
					 ---------
2022-07-21 17:16:50,040 fedbiomed INFO - TRAINING 
					 NODE_ID: node_15a68d33-1a5d-408a-959c-042c1287feab 
					 Epoch: 2 | Completed: 64/156 (40%) 
 					 Loss: 0.162127 
					 ---------
2022-07-21 17:16:50,060 fedbiomed INFO - TRAINING 
					 NODE_ID: node_089d07d3-095a-48d7-b49a-f55edc79307e 
					 Epoch: 2 | Completed: 160/248 (62%) 
 					 Loss: 0.311071 
					 ---------
2022-07-21 17:16:50,079 fedbiomed INFO - TRAININ

2022-07-21 17:16:50,607 fedbiomed INFO - INFO
					 NODE node_15a68d33-1a5d-408a-959c-042c1287feab
					 MESSAGE: results uploaded successfully 
-----------------------------------------------------------------
2022-07-21 17:16:50,644 fedbiomed INFO - TRAINING 
					 NODE_ID: node_089d07d3-095a-48d7-b49a-f55edc79307e 
					 Epoch: 4 | Completed: 224/248 (88%) 
 					 Loss: 0.741301 
					 ---------
2022-07-21 17:16:50,653 fedbiomed INFO - TRAINING 
					 NODE_ID: node_089d07d3-095a-48d7-b49a-f55edc79307e 
					 Epoch: 4 | Completed: 128/248 (100%) 
 					 Loss: 0.172706 
					 ---------
2022-07-21 17:16:50,678 fedbiomed INFO - TRAINING 
					 NODE_ID: node_089d07d3-095a-48d7-b49a-f55edc79307e 
					 Epoch: 5 | Completed: 32/248 (12%) 
 					 Loss: 0.070958 
					 ---------
2022-07-21 17:16:50,709 fedbiomed INFO - TRAINING 
					 NODE_ID: node_089d07d3-095a-48d7-b49a-f55edc79307e 
					 Epoch: 5 | Completed: 64/248 (25%) 
 					 Loss: 0.017798 
					 ---------
2022-07-21 17:16:50,739 fe

1

### 4. Fed-Synthetic

In [37]:
from flamby.datasets.fed_synthetic import (Baseline as sy_baseline,
                                           BaselineLoss as sy_baseline_loss,
                                           Optimizer as sy_optimizer,
                                           BATCH_SIZE as sy_batch_size,
                                           LR as sy_lr)
from fedbiomed.common.training_plans import TorchTrainingPlan
from torch.optim import Adam

class FedSyntheticTrainingPlan(TorchTrainingPlan):
    def __init__(self, model_args: dict = {}):
        super(FedSyntheticTrainingPlan, self).__init__(model_args)
        
        self.model = sy_baseline(model_args.get('input_dim', 10), model_args.get('output_dim', 1)) # specific to synthetic use case
        self.loss = sy_baseline_loss()
        
        deps = ['from torch.optim import Adam',
               'from flamby.datasets.fed_synthetic import (Baseline as sy_baseline,\
                BaselineLoss as sy_baseline_loss,\
                Optimizer as sy_optimizer)',]
        self.add_dependency(deps)
        
        self.optimizer = sy_optimizer(self.parameters())
    
    def training_step(self, data, target):
        #this function must return the loss to backward it 
        output = self.model(data)
        loss = self.loss(output, target)
        return loss

In [38]:
training_args = {
    'batch_size': sy_batch_size,
    'lr': sy_lr,
    'epochs': 5,
}

model_args = {
    'input_dim': 10,
    'output_dim': 1,
}

In [39]:
from fedbiomed.researcher.experiment import Experiment
from fedbiomed.researcher.aggregators.fedavg import FedAverage

tags =  ['sy']
num_rounds = 1

exp = Experiment(tags=tags,
                 model_args=model_args,
                 model_class=FedSyntheticTrainingPlan,
                 training_args=training_args,
                 round_limit=num_rounds,
                 aggregator=FedAverage(),
                )

2022-07-20 16:27:33,588 fedbiomed INFO - Searching dataset with data tags: ['sy'] for all nodes
2022-07-20 16:27:43,602 fedbiomed INFO - Node selected for training -> node_1eb02619-83fa-4523-8c2d-259ed6835d63
2022-07-20 16:27:43,603 fedbiomed INFO - Node selected for training -> node_8e6bb63e-8a5a-41c1-94a0-3c980435dbb9
2022-07-20 16:27:43,606 fedbiomed INFO - Checking data quality of federated datasets...
2022-07-20 16:27:43,614 fedbiomed DEBUG - Model file has been saved: /home/ayed/fedbiomed/var/experiments/Experiment_0016/my_model_40d12b12-1018-4415-b39a-47e489127245.py
2022-07-20 16:27:43,639 fedbiomed DEBUG - upload (HTTP POST request) of file /home/ayed/fedbiomed/var/experiments/Experiment_0016/my_model_40d12b12-1018-4415-b39a-47e489127245.py successful, with status code 201
2022-07-20 16:27:43,657 fedbiomed DEBUG - upload (HTTP POST request) of file /home/ayed/fedbiomed/var/experiments/Experiment_0016/aggregated_params_init_58080bc7-4998-4512-a188-2e07e62fd02c.pt successful, wi

In [40]:
exp.run()

2022-07-20 16:27:55,272 fedbiomed INFO - Sampled nodes in round 0 ['node_1eb02619-83fa-4523-8c2d-259ed6835d63', 'node_8e6bb63e-8a5a-41c1-94a0-3c980435dbb9']
2022-07-20 16:27:55,274 fedbiomed INFO - Sending request 
					 To: node_1eb02619-83fa-4523-8c2d-259ed6835d63 
					 Request: : Perform training with the arguments: {'researcher_id': 'researcher_dfd1d299-4cc6-49aa-8623-b8b8b842003c', 'job_id': 'd3e33f31-663e-4574-abae-4ba0f8bf06dd', 'training_args': scheme:
{'lr': {'rules': [<class 'float'>, <function TrainingArgs._lr_hook at 0x7f5e4f623af0>], 'required': False}, 'batch_size': {'rules': [<class 'int'>], 'required': False}, 'epochs': {'rules': [<class 'int'>], 'required': False}, 'dry_run': {'rules': [<class 'bool'>], 'required': False}, 'batch_maxnum': {'rules': [<class 'int'>], 'required': False}, 'test_ratio': {'rules': [<class 'float'>, <function TrainingArgs._test_ratio_hook at 0x7f5e4f6239d0>], 'required': False, 'default': 0.0}, 'test_on_local_updates': {'rules': [<class 'boo

2022-07-20 16:27:55,560 fedbiomed INFO - TRAINING 
					 NODE_ID: node_8e6bb63e-8a5a-41c1-94a0-3c980435dbb9 
					 Epoch: 1 | Completed: 32/110 (29%) 
 					 Loss: 1.076702 
					 ---------
2022-07-20 16:27:55,561 fedbiomed INFO - TRAINING 
					 NODE_ID: node_8e6bb63e-8a5a-41c1-94a0-3c980435dbb9 
					 Epoch: 1 | Completed: 64/110 (57%) 
 					 Loss: 5.305909 
					 ---------
2022-07-20 16:27:55,563 fedbiomed INFO - TRAINING 
					 NODE_ID: node_8e6bb63e-8a5a-41c1-94a0-3c980435dbb9 
					 Epoch: 1 | Completed: 96/110 (86%) 
 					 Loss: 4.534348 
					 ---------
2022-07-20 16:27:55,564 fedbiomed INFO - TRAINING 
					 NODE_ID: node_8e6bb63e-8a5a-41c1-94a0-3c980435dbb9 
					 Epoch: 2 | Completed: 32/110 (29%) 
 					 Loss: 4.821252 
					 ---------
2022-07-20 16:27:55,566 fedbiomed INFO - TRAINING 
					 NODE_ID: node_8e6bb63e-8a5a-41c1-94a0-3c980435dbb9 
					 Epoch: 2 | Completed: 64/110 (57%) 
 					 Loss: 3.204389 
					 ---------
2022-07-20 16:27:55,569 fedbiomed INFO - TRAINING 

1

2022-07-20 16:28:36,085 fedbiomed INFO - CRITICAL
					 NODE node_1eb02619-83fa-4523-8c2d-259ed6835d63
					 MESSAGE: Node stopped in signal_handler, probably by user decision (Ctrl C)
-----------------------------------------------------------------
2022-07-20 16:28:37,866 fedbiomed INFO - CRITICAL
					 NODE node_8e6bb63e-8a5a-41c1-94a0-3c980435dbb9
					 MESSAGE: Node stopped in signal_handler, probably by user decision (Ctrl C)
-----------------------------------------------------------------
2022-07-20 16:32:20,382 fedbiomed INFO - INFO
					 NODE node_8e6bb63e-8a5a-41c1-94a0-3c980435dbb9
					 MESSAGE: Starting task manager
-----------------------------------------------------------------
2022-07-20 16:32:37,282 fedbiomed INFO - INFO
					 NODE node_1eb02619-83fa-4523-8c2d-259ed6835d63
					 MESSAGE: Starting task manager
-----------------------------------------------------------------


### 5. Fed-ISIC

In [3]:
from flamby.datasets.fed_isic2019 import (Baseline as is_baseline,
                                          BaselineLoss as is_baseline_loss,
                                          Optimizer as is_optimizer,
                                          BATCH_SIZE as is_batch_size,
                                          LR as is_lr
                                         )
from fedbiomed.common.training_plans import TorchTrainingPlan
from torch.optim import Adam

class FedISICTrainingPlan(TorchTrainingPlan):
    def __init__(self, model_args: dict = {}):
        super(FedISICTrainingPlan, self).__init__(model_args)
        
        self.model = is_baseline()
        self.loss = is_baseline_loss()
        
        deps = ['from torch.optim import Adam',
               'from flamby.datasets.fed_isic2019 import (Baseline as is_baseline,\
                BaselineLoss as is_baseline_loss,\
                Optimizer as is_optimizer)',]
        self.add_dependency(deps)
        
        self.optimizer = Adam(self.parameters())
    
    def training_step(self, data, target):
        output = self.model(data)
        loss = self.loss(output, target)
        return loss

In [4]:
training_args = {
    'batch_size': is_batch_size,
    'lr': is_lr,
    'epochs': 1,
}

In [5]:
from fedbiomed.researcher.experiment import Experiment
from fedbiomed.researcher.aggregators.fedavg import FedAverage

tags =  ['is']
num_rounds = 1

exp = Experiment(tags=tags,
                 model_args=model_args,
                 model_class=FedISICTrainingPlan,
                 training_args=training_args,
                 round_limit=num_rounds,
                 aggregator=FedAverage(),
                )

2022-07-20 17:19:32,420 fedbiomed INFO - Component environment:
2022-07-20 17:19:32,421 fedbiomed INFO - type = ComponentType.RESEARCHER
2022-07-20 17:19:32,781 fedbiomed INFO - Messaging researcher_3d2e8dbb-47c9-4e58-849b-073f78b493b9 successfully connected to the message broker, object = <fedbiomed.common.messaging.Messaging object at 0x7f56a5a784f0>
2022-07-20 17:19:32,809 fedbiomed INFO - Searching dataset with data tags: ['is'] for all nodes
2022-07-20 17:19:42,821 fedbiomed INFO - Node selected for training -> node_a17c8a69-83ec-4617-a920-4f8a5362e8b1
2022-07-20 17:19:42,822 fedbiomed INFO - Node selected for training -> node_a10eca4b-d2f5-4a4a-961b-3bd91352f121
2022-07-20 17:19:42,825 fedbiomed INFO - Checking data quality of federated datasets...
2022-07-20 17:19:42,970 fedbiomed DEBUG - Model file has been saved: /home/ayed/fedbiomed/var/experiments/Experiment_0000/my_model_36c9d40d-edc0-4464-b9f2-fcb252b3561d.py
2022-07-20 17:19:43,115 fedbiomed DEBUG - upload (HTTP POST requ

Loaded pretrained weights for efficientnet-b0
Number of features output by EfficientNet 1280


2022-07-20 17:19:43,536 fedbiomed DEBUG - upload (HTTP POST request) of file /home/ayed/fedbiomed/var/experiments/Experiment_0000/aggregated_params_init_c7a090e0-dab5-4cdc-8599-fced2dae3c9b.pt successful, with status code 201


In [ ]:
exp.run()

2022-07-20 17:19:47,662 fedbiomed INFO - Sampled nodes in round 0 ['node_a17c8a69-83ec-4617-a920-4f8a5362e8b1', 'node_a10eca4b-d2f5-4a4a-961b-3bd91352f121']
2022-07-20 17:19:47,664 fedbiomed INFO - Sending request 
					 To: node_a17c8a69-83ec-4617-a920-4f8a5362e8b1 
					 Request: : Perform training with the arguments: {'researcher_id': 'researcher_3d2e8dbb-47c9-4e58-849b-073f78b493b9', 'job_id': '9a0b8884-aada-4853-ba76-d43b812d7c91', 'training_args': scheme:
{'lr': {'rules': [<class 'float'>, <function TrainingArgs._lr_hook at 0x7f56a65e7430>], 'required': False}, 'batch_size': {'rules': [<class 'int'>], 'required': False}, 'epochs': {'rules': [<class 'int'>], 'required': False}, 'dry_run': {'rules': [<class 'bool'>], 'required': False}, 'batch_maxnum': {'rules': [<class 'int'>], 'required': False}, 'test_ratio': {'rules': [<class 'float'>, <function TrainingArgs._test_ratio_hook at 0x7f56a65e7310>], 'required': False, 'default': 0.0}, 'test_on_local_updates': {'rules': [<class 'boo

2022-07-20 17:20:56,045 fedbiomed INFO - TRAINING 
					 NODE_ID: node_a10eca4b-d2f5-4a4a-961b-3bd91352f121 
					 Epoch: 1 | Completed: 16/3163 (1%) 
 					 Loss: 0.433575 
					 ---------
2022-07-20 17:21:03,994 fedbiomed INFO - TRAINING 
					 NODE_ID: node_a17c8a69-83ec-4617-a920-4f8a5362e8b1 
					 Epoch: 1 | Completed: 18/9930 (0%) 
 					 Loss: 22.162544 
					 ---------
2022-07-20 17:21:06,121 fedbiomed INFO - TRAINING 
					 NODE_ID: node_a10eca4b-d2f5-4a4a-961b-3bd91352f121 
					 Epoch: 1 | Completed: 18/3163 (1%) 
 					 Loss: 0.507334 
					 ---------
2022-07-20 17:21:11,942 fedbiomed INFO - TRAINING 
					 NODE_ID: node_a17c8a69-83ec-4617-a920-4f8a5362e8b1 
					 Epoch: 1 | Completed: 20/9930 (0%) 
 					 Loss: 4.105480 
					 ---------
2022-07-20 17:21:15,180 fedbiomed INFO - TRAINING 
					 NODE_ID: node_a10eca4b-d2f5-4a4a-961b-3bd91352f121 
					 Epoch: 1 | Completed: 20/3163 (1%) 
 					 Loss: 0.236378 
					 ---------
2022-07-20 17:21:20,742 fedbiomed INFO - TRAINING

2022-07-20 17:23:40,966 fedbiomed INFO - TRAINING 
					 NODE_ID: node_a10eca4b-d2f5-4a4a-961b-3bd91352f121 
					 Epoch: 1 | Completed: 56/3163 (2%) 
 					 Loss: 0.119603 
					 ---------
2022-07-20 17:23:46,336 fedbiomed INFO - TRAINING 
					 NODE_ID: node_a17c8a69-83ec-4617-a920-4f8a5362e8b1 
					 Epoch: 1 | Completed: 58/9930 (1%) 
 					 Loss: 25.647467 
					 ---------
2022-07-20 17:23:49,056 fedbiomed INFO - TRAINING 
					 NODE_ID: node_a10eca4b-d2f5-4a4a-961b-3bd91352f121 
					 Epoch: 1 | Completed: 58/3163 (2%) 
 					 Loss: 0.307686 
					 ---------
2022-07-20 17:23:54,172 fedbiomed INFO - TRAINING 
					 NODE_ID: node_a17c8a69-83ec-4617-a920-4f8a5362e8b1 
					 Epoch: 1 | Completed: 60/9930 (1%) 
 					 Loss: 173.675964 
					 ---------
2022-07-20 17:23:57,165 fedbiomed INFO - TRAINING 
					 NODE_ID: node_a10eca4b-d2f5-4a4a-961b-3bd91352f121 
					 Epoch: 1 | Completed: 60/3163 (2%) 
 					 Loss: 1.874871 
					 ---------
2022-07-20 17:24:02,452 fedbiomed INFO - TRAINI

2022-07-20 17:26:14,031 fedbiomed INFO - TRAINING 
					 NODE_ID: node_a10eca4b-d2f5-4a4a-961b-3bd91352f121 
					 Epoch: 1 | Completed: 96/3163 (3%) 
 					 Loss: 1.239231 
					 ---------
2022-07-20 17:26:16,337 fedbiomed INFO - TRAINING 
					 NODE_ID: node_a17c8a69-83ec-4617-a920-4f8a5362e8b1 
					 Epoch: 1 | Completed: 98/9930 (1%) 
 					 Loss: 13.841805 
					 ---------
2022-07-20 17:26:20,982 fedbiomed INFO - TRAINING 
					 NODE_ID: node_a10eca4b-d2f5-4a4a-961b-3bd91352f121 
					 Epoch: 1 | Completed: 98/3163 (3%) 
 					 Loss: 0.978710 
					 ---------
2022-07-20 17:26:23,464 fedbiomed INFO - TRAINING 
					 NODE_ID: node_a17c8a69-83ec-4617-a920-4f8a5362e8b1 
					 Epoch: 1 | Completed: 100/9930 (1%) 
 					 Loss: 8.376694 
					 ---------
2022-07-20 17:26:27,813 fedbiomed INFO - TRAINING 
					 NODE_ID: node_a10eca4b-d2f5-4a4a-961b-3bd91352f121 
					 Epoch: 1 | Completed: 100/3163 (3%) 
 					 Loss: 2.608762 
					 ---------
2022-07-20 17:26:30,328 fedbiomed INFO - TRAINI

2022-07-20 17:28:35,637 fedbiomed INFO - TRAINING 
					 NODE_ID: node_a10eca4b-d2f5-4a4a-961b-3bd91352f121 
					 Epoch: 1 | Completed: 136/3163 (4%) 
 					 Loss: 3.713423 
					 ---------
2022-07-20 17:28:39,185 fedbiomed INFO - TRAINING 
					 NODE_ID: node_a17c8a69-83ec-4617-a920-4f8a5362e8b1 
					 Epoch: 1 | Completed: 138/9930 (1%) 
 					 Loss: 3.020479 
					 ---------
2022-07-20 17:28:42,037 fedbiomed INFO - TRAINING 
					 NODE_ID: node_a10eca4b-d2f5-4a4a-961b-3bd91352f121 
					 Epoch: 1 | Completed: 138/3163 (4%) 
 					 Loss: 5.157810 
					 ---------
2022-07-20 17:28:45,837 fedbiomed INFO - TRAINING 
					 NODE_ID: node_a17c8a69-83ec-4617-a920-4f8a5362e8b1 
					 Epoch: 1 | Completed: 140/9930 (1%) 
 					 Loss: 9.656112 
					 ---------
2022-07-20 17:28:48,599 fedbiomed INFO - TRAINING 
					 NODE_ID: node_a10eca4b-d2f5-4a4a-961b-3bd91352f121 
					 Epoch: 1 | Completed: 140/3163 (4%) 
 					 Loss: 3.792548 
					 ---------
2022-07-20 17:28:51,983 fedbiomed INFO - TRAI

2022-07-20 17:30:51,200 fedbiomed INFO - TRAINING 
					 NODE_ID: node_a10eca4b-d2f5-4a4a-961b-3bd91352f121 
					 Epoch: 1 | Completed: 176/3163 (6%) 
 					 Loss: 1.252156 
					 ---------
2022-07-20 17:30:53,947 fedbiomed INFO - TRAINING 
					 NODE_ID: node_a17c8a69-83ec-4617-a920-4f8a5362e8b1 
					 Epoch: 1 | Completed: 178/9930 (2%) 
 					 Loss: 7.058232 
					 ---------
2022-07-20 17:30:57,559 fedbiomed INFO - TRAINING 
					 NODE_ID: node_a10eca4b-d2f5-4a4a-961b-3bd91352f121 
					 Epoch: 1 | Completed: 178/3163 (6%) 
 					 Loss: 0.577924 
					 ---------
2022-07-20 17:31:00,053 fedbiomed INFO - TRAINING 
					 NODE_ID: node_a17c8a69-83ec-4617-a920-4f8a5362e8b1 
					 Epoch: 1 | Completed: 180/9930 (2%) 
 					 Loss: 10.695940 
					 ---------
2022-07-20 17:31:03,888 fedbiomed INFO - TRAINING 
					 NODE_ID: node_a10eca4b-d2f5-4a4a-961b-3bd91352f121 
					 Epoch: 1 | Completed: 180/3163 (6%) 
 					 Loss: 11.637127 
					 ---------
2022-07-20 17:31:06,471 fedbiomed INFO - TR

In [ ]:
#12.62